In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive


Mounted at /content/gdrive
'Colab Notebooks'      'Internship-Aug2020-Gowtham R.pdf'   Photos
'Getting started.pdf'  'My Drive'			   'Pongal 2021'


In [3]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project

/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project


In [4]:
from keras.models import load_model
import pandas as pd
import numpy as np 
from os.path import join
from myFunctions.candle_stick_plot import plotCandleStick
from myFunctions.list_data_dir import load_data_files
from myFunctions import dataConversion 
from myFunctions.predictionPlot import prediction_plot
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.model_selection import RandomizedSearchCV,KFold

In [5]:
fields = ['Date','High','Low','Open','Close']
lookback = 30
datafolder = r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50"
prediction_date = pd.Timestamp(2020,11,30,00,00,00)

In [6]:
df = pd.read_csv(r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50/ADANIPORTS.csv",usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close
0,2007-11-27,770.00,1050.00,770.0,962.90
1,2007-11-28,984.00,990.00,874.0,893.90
2,2007-11-29,909.00,914.75,841.0,884.20
3,2007-11-30,890.00,958.00,890.0,921.55
4,2007-12-03,939.75,995.00,922.0,969.30


In [8]:
High_X , High_y = dataConversion.convert2matrix(df['High'],lookback)

In [10]:
High_X_train, High_X_test , High_y_train , High_y_test = train_test_split(High_X,High_y,test_size=0.2)

In [11]:
def create_model(layers=1 ,neurons=256):
    model=Sequential()

    model.add(Dense(500,activation='relu',input_shape=(lookback,)))

    for i in range(layers):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='relu'))
    model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
    return model

In [13]:
model = KerasRegressor(build_fn=create_model,epochs=5,batch_size=16)
params = dict(layers=[1,2,3,4,5],neurons=[5,25,50,100,150,200,250,300])
random_search = RandomizedSearchCV(model,params,cv=KFold(3))
random_search.fit(High_X_train,High_y_train)

Epoch 1/5
107/107 [==============================] - 1s 2ms/step - loss: 17333.4602 - mse: 17333.4602
Epoch 2/5
107/107 [==============================] - 0s 2ms/step - loss: 1612.4246 - mse: 1612.4246
Epoch 3/5
107/107 [==============================] - 0s 2ms/step - loss: 780.4967 - mse: 780.4967
Epoch 4/5
107/107 [==============================] - 0s 2ms/step - loss: 777.8472 - mse: 777.8472
Epoch 5/5
54/54 [==============================] - 0s 1ms/step - loss: 377.5019 - mse: 377.5019
Epoch 1/5
107/107 [==============================] - 1s 2ms/step - loss: 76504.7874 - mse: 76504.7874
Epoch 2/5
107/107 [==============================] - 0s 2ms/step - loss: 949.0369 - mse: 949.0369
Epoch 3/5
107/107 [==============================] - 0s 2ms/step - loss: 702.3976 - mse: 702.3976
Epoch 4/5
107/107 [==============================] - 0s 2ms/step - loss: 465.8948 - mse: 465.8948
Epoch 5/5
54/54 [==============================] - 0s 1ms/step - loss: 1654.1807 - mse: 1654.1807
Epoch 1/5
10

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f69ff5d8590>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'layers': [1, 2, 3, 4, 5],
                                        'neurons': [5, 25, 50, 100, 150, 200,
                                                    250, 300]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [14]:
random_search.best_params_

{'layers': 4, 'neurons': 50}

In [16]:
random_search.best_score_

-769.9719848632812

In [17]:
best_high_model = random_search.best_estimator_

In [19]:
best_high_model.model.evaluate(High_X_test,High_y_test)

20/20 [==============================] - 0s 1ms/step - loss: 362.2304 - mse: 362.2304


[362.2304382324219, 362.2304382324219]

In [20]:
best_high_model.model.save("/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/high_model")

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/high_model/assets
